<a href="https://colab.research.google.com/github/arthurflor23/computer-vision/blob/master/HW03/HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment#3

In this assignment you will practice putting together a simple image classification pipeline, based on the k-Nearest Neighbor or the SVM/Softmax classifier. Once you have the starter code (regardless of which method you choose above), you will need to download the CIFAR-10 dataset.

**NOTES**
1. The assignment#3 code has been tested to be compatible with python version 3.7. You will need to make sure that during your virtual environment setup that the correct version of python is used. You can confirm your python version by (1) activating your virtualenv and (2) running which python.

2. Feel free to pick any others ways to implement this assignment#3.

## Image Classifier (CIFAR-10)

First, install TensorFlow 2.0:

In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 58kB/s 
     |████████████████████████████████| 419kB 51.8MB/s 
     |████████████████████████████████| 3.0MB 42.8MB/s 
     |████████████████████████████████| 61kB 28.9MB/s 


Setup the training dataset using the Dataset API in TensorFlow and extracting the CIFAR10 data from the Keras datasets library:

In [2]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# treatment to the train dataset, with batch size 32 and random distortion of the images:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32), 255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()

# treatment to the validation dataset, with a larger batch size and no random distortion of the images
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1000).shuffle(10000)
valid_dataset = valid_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32),255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
valid_dataset = valid_dataset.repeat()

170500096/170498071 [==============================] - 2s 0us/step


Create the convolutional neural network to classify the images:

In [0]:
class CIFAR10Model(tf.keras.Model):

    def __init__(self):
        super(CIFAR10Model, self).__init__(name='cifar_cnn')
        
        self.conv1 = tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.max_pool2d = tf.keras.layers.MaxPooling2D((3, 3), (2, 2), padding='same')
        self.max_norm = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.001))

        self.flatten = tf.keras.layers.Flatten()

        self.fc1 = tf.keras.layers.Dense(750, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.fc2 = tf.keras.layers.Dense(10)
        self.softmax = tf.keras.layers.Softmax()

    def call(self, x):
        x = self.max_pool2d(self.conv1(x))
        x = self.max_norm(x)
        x = self.max_pool2d(self.conv2(x))
        x = self.max_norm(x)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

Instantiate the model class, compile and run the training.

**Note**: Also added an early stopping with 5 patience for the *val_loss*.

In [4]:
model = CIFAR10Model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              min_delta=0.001,
                                              patience=5,
                                              restore_best_weights=True, 
                                              verbose=1)]

model.fit(train_dataset,
          epochs=1000,
          steps_per_epoch=1500, 
          validation_data=valid_dataset,
          validation_steps=3,
          callbacks=callbacks,
          shuffle=True,
          verbose=1)

Epoch 1/1000
1500/1500 [==============================] - 25s 17ms/step - loss: 2.4422 - accuracy: 0.3812 - val_loss: 11.3696 - val_accuracy: 0.1377
Epoch 2/1000
1500/1500 [==============================] - 18s 12ms/step - loss: 1.6970 - accuracy: 0.5812 - val_loss: 3.0047 - val_accuracy: 0.2493
Epoch 3/1000
1500/1500 [==============================] - 19s 12ms/step - loss: 1.5668 - accuracy: 0.6403 - val_loss: 3.1246 - val_accuracy: 0.2020
Epoch 4/1000
1500/1500 [==============================] - 19s 12ms/step - loss: 1.4901 - accuracy: 0.6706 - val_loss: 3.5717 - val_accuracy: 0.2023
Epoch 5/1000
1500/1500 [==============================] - 18s 12ms/step - loss: 1.4457 - accuracy: 0.6803 - val_loss: 4.4993 - val_accuracy: 0.2010
Epoch 6/1000
1500/1500 [==============================] - 18s 12ms/step - loss: 1.4121 - accuracy: 0.6937 - val_loss: 2.7962 - val_accuracy: 0.3430
Epoch 7/1000
1500/1500 [==============================] - 19s 12ms/step - loss: 1.3776 - accuracy: 0.7069 - val